In [ ]:
import ROOT
from sys import exit
import numpy as np
from numpy.testing import assert_array_equal
from nose.tools import (raises, assert_raises, assert_true,
                        assert_equal, assert_almost_equal)
import root_numpy as rnp
import pandas as pd
from pandas.plotting import scatter_matrix
ROOT.ROOT.EnableImplicitMT()

from sklearn import preprocessing

import torch as torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

#look at data loaded
import matplotlib.pyplot as plt
import plotly.express as px

#read in key parameters from root file
#data = rnp.root2array("../data_02/mlTree.root","mltree",branches=['x','e1','e2','e3','e12','e13',
#                                                                   'e23','e123','recoilID'])

data_in = rnp.root2array("../data/pyTreeAverageFat.root","pytree",
                         branches=['e0','e1','e2','e3','e4','e5','e6','x','m','dt','gmult','ge','gid','glabel'])
assert_equal(data_in.dtype.names, ('e0','e1','e2','e3','e4','e5','e6','x','m','dt','gmult','ge','gid','glabel'))
print(data_in.dtype)
num_data_columns=14.0
#convert to pandas
df = pd.DataFrame(data=data_in)
print(df)
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
x_scaled = min_max_scaler.fit_transform(x)
res_df = pd.DataFrame(x_scaled)
df_g1293 = df[(df['ge'] > 1285) & (df['ge'] < 1298)]
print(df_g1293.size/num_data_columns)
#data to torch tensor
X = torch.from_numpy(res_df.values).float().to(torch.device('cpu'))
#print(X)

class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(20, 10)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(10, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        a2 = self.fc2(dout)
        h2 = self.prelu(a2)
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y
    
net = Net()

In [ ]:
#Model D with AverageFatD_train data (mar31 9pm)
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(11, 40)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(40, 20)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(20, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        a2 = self.fc2(dout)
        h2 = self.prelu(a2)
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y
    
net = Net()

In [ ]:
#print(net.parameters)
#print(net.state_dict())
save_path="./model_saves/model_D.pt"
net = torch.load(save_path)
net.eval()

In [ ]:
with torch.no_grad():
    result = net(X[0:6000000,0:11])
    #result = torch.round(result)
#plt.plot(result.data)
print(torch.max(result))
print(torch.min(result))
torch.histc(result)
#print(result.data)

df_F = pd.DataFrame(data=result.numpy())
#print(df_F)
df_F.columns = ["A"]

#fig = px.histogram(df_F,x="A",nbins=100)
#fig.update_xaxes(range=[0.0,1.0])
#fig.show()

In [ ]:
#pull out events from model decision
result_df = pd.concat([df, df_F], axis=1)
#dump to csv
#result_df.to_csv("XXXX.csv",sep=" ",header=False)
#print(result_df)
result_df.to_csv("output/pyTreeAverageFatD_tester.csv",sep=" ",header=False)





In [ ]:

s38_low=0.75
other_high=0.3
num_data_columns=15
s38 = result_df[(result_df['A'] > s38_low) & (result_df['A'] < 1)]
cl38 = result_df[(result_df['A'] < other_high) & (result_df['A'] < 1)]
misc = result_df[(result_df['A'] > other_high) & (result_df['A'] < s38_low)]
g1293a = result_df[(result_df['A'] > s38_low) & (result_df['ge'] >1285) & (result_df['ge'] < 1298)]
g1293_s38 = g1293a.size/num_data_columns
g1293b = result_df[(result_df['A'] < other_high) & (result_df['ge'] >1285) & (result_df['ge'] < 1298)]
g1293_cl38 = g1293b.size/num_data_columns
g1293c = result_df[(result_df['A'] > other_high) & (result_df['A'] < s38_low) & 
                   (result_df['ge'] >1285) & (result_df['ge'] < 1298)]
g1293_misc = g1293c.size/num_data_columns
#print(s38.size/num_data_columns)
print("Total 1293 gamma rays for 38S: %d" 
      % (g1293_s38))
plt.figure(figsize=(10,4))
fig = px.histogram(s38, x="ge", nbins=10000, color = "glabel")
fig.update_xaxes(range=[100, 3100])
fig.show()

plt.figure(figsize=(10,4))
fig = px.histogram(cl38, x="ge", nbins=10000, color = "glabel")
fig.update_xaxes(range=[100, 3100])
fig.show()

plt.figure(figsize=(10,4))
fig = px.histogram(misc, x="ge", nbins=10000, color = "glabel")
fig.update_xaxes(range=[100, 3100])
fig.show()
